<a href="https://colab.research.google.com/github/alocken/Project-4-LLM-Project-Activity---Topic-Modeling/blob/main/Project_4_2_representation_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 4: LLM Project Activity - Topic Modeling**




### **Week 23** Project 4: 2-Representation
*   Apply tokenization and text representation methods in the project. (9.3)

In [ ]:
#Bag of Words (BoW) selected to represent text
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

# Fit on training data and transform both train and test sets
X_train_bow = vectorizer.fit_transform(ds_train['text'])
X_test_bow = vectorizer.transform(ds_test['text'])

# Convert to dense DataFrame for inspection
bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())

print(bow_df.head())

   aa  aaa  aaaaaaaaaaaa  \
0   0    0             0   
1   0    0             0   
2   0    0             0   
3   0    0             0   
4   0    0             0   

   aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg  \
0                                                  0                                 
1                                                  0                                 
2                                                  0                                 
3                                                  0                                 
4                                                  0                                 

   aaah  aaahh  aaahhhh  aaai  aaaimit  aaamajors  ...  zwakke  zware  zwarte  \
0     0      0        0     0        0          0  ...       0      0       0   
1     0      0        0     0        0          0  ...       0      0       0   
2     0      0        0     0        0          0  ...       0      0  

In [ ]:
# confirm full preprocessing complete with tokenization for text
import pandas as pd
import string
from nltk.corpus import stopwords

# Ensure stopwords downloaded
import nltk
nltk.download('stopwords')

# Define stopwords set
stop_words = set(stopwords.words('english'))

# Define a preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize using simple split
    tokens = text.split()

    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]

    return tokens  # Return list of tokens

# Apply to dataset
ds_train['tokens'] = ds_train['text'].apply(preprocess_text)
ds_test['tokens'] = ds_test['text'].apply(preprocess_text)

# print a sample
print(ds_train[['text', 'tokens']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  \
0  wondering anyone could enlighten car saw day s...   
1  fair number brave souls upgraded si clock osci...   
2  well folks mac plus finally gave ghost weekend...   
3  weiteks addressphone number id like get inform...   
4  article tombakerworldstdcom tom baker understa...   

                                              tokens  
0  [wondering, anyone, could, enlighten, car, saw...  
1  [fair, number, brave, souls, upgraded, si, clo...  
2  [well, folks, mac, plus, finally, gave, ghost,...  
3  [weiteks, addressphone, number, id, like, get,...  
4  [article, tombakerworldstdcom, tom, baker, und...  


In [ ]:
# documents, total words in dataset
# Number of documents (rows)
num_docs_train = len(ds_train)
num_docs_test = len(ds_test)
total_docs = num_docs_train + num_docs_test

print(f"Total number of documents: {total_docs}")

# Flatten the token lists from both train and test sets
all_tokens = ds_train['tokens'].explode().tolist() + ds_test['tokens'].explode().tolist()

# Get the vocabulary (unique words)
vocab = set(all_tokens)
vocab_size = len(vocab)

print(f"Total number of unique words in the vocabulary: {vocab_size}")


Total number of documents: 18846
Total number of unique words in the vocabulary: 98160


In [ ]:
# build ML model using BoW (vectorize toeknized data) and logistic regresssion (baseline model for text classfication
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

# Join tokens into strings for BoW
X_train = ds_train['tokens'].apply(lambda tokens: ' '.join(tokens))
X_test = ds_test['tokens'].apply(lambda tokens: ' '.join(tokens))

y_train = ds_train['label']
y_test = ds_test['label']

# Create a pipeline: BoW + Logistic Regression
model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression(max_iter=1000))])

In [ ]:
#evaluate model by testing aginst test data and evaluate outputs summary report

# Predict on test set
y_pred = model.predict(X_test)

# Show evaluation report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.42      0.44       319
           1       0.53      0.61      0.57       389
           2       0.58      0.58      0.58       394
           3       0.53      0.52      0.53       392
           4       0.63      0.62      0.63       385
           5       0.73      0.59      0.66       395
           6       0.74      0.76      0.75       390
           7       0.62      0.65      0.63       396
           8       0.63      0.71      0.67       398
           9       0.48      0.81      0.60       397
          10       0.83      0.79      0.81       399
          11       0.76      0.63      0.69       396
          12       0.50      0.49      0.50       393
          13       0.72      0.62      0.67       396
          14       0.65      0.64      0.65       394
          15       0.69      0.63      0.66       398
          16       0.50      0.54      0.52       364
          17       0.73    

**Model Performance Summary**
Overall Accuracy: 61%
F1-scores: vary significantly across classes (e.g., classes 10,6 have good scores of ~.75-.81, other classes 18,9 low scores ~.31-.38)

Limitation: this could indicate a potential for class imbalance as some classes may have fewer samples thatn others (e.g., class 19 with 251 vs. class 2 with 394)




In [ ]:
# attempt to improve model performnce by using TF-IDF instead of CountVectorizer to down-weight common words

from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])
model.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression(max_iter=1000))])

In [ ]:
#evaluate model by testing aginst test data and evaluate outputs summary report
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6727296866702072

Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.44      0.46       319
           1       0.58      0.68      0.63       389
           2       0.65      0.61      0.63       394
           3       0.64      0.61      0.62       392
           4       0.75      0.66      0.70       385
           5       0.78      0.69      0.73       395
           6       0.74      0.78      0.76       390
           7       0.75      0.68      0.71       396
           8       0.47      0.80      0.59       398
           9       0.82      0.79      0.81       397
          10       0.88      0.87      0.87       399
          11       0.86      0.66      0.74       396
          12       0.53      0.58      0.55       393
          13       0.76      0.76      0.76       396
          14       0.70      0.74      0.72       394
          15       0.63      0.78      0.70       398
          16       0.58    

**Model Performance Summary: With Attempted Improvements**

Overall Accuracy: demonstrated improvment with increase from 61% to 67%

F1-scores: improvements noted as most classes demonstrating f1-score in range of ~.60-.80

Limitation: some classes still underperforming (e.g., class 0 and 19 with f1 scores under .50.